# Workshop for programatically creating Exit Questionnaire payloads and sending to the CIP API

## Documentation for Gel Report Models is available: http://gelreportmodels.genomicsengland.co.uk

### We are specifically interested in the Version 3 Rare Disease Exit Questionnaire: http://gelreportmodels.genomicsengland.co.uk/html_schemas/org.gel.models.report.avro/3.0.0/ExitQuestionnaire.html#/schema/org.gel.models.report.avro.RareDiseaseExitQuestionnaire

### Generating an Exit Questionnaire

In [1]:
from protocols.reports_3_0_0 import RareDiseaseExitQuestionnaire as EQ

In [2]:
eq = EQ()

The `toJsonDict` method allows us to inspect a json representation of what the object contains

In [3]:
from pprint import pprint
pprint(eq.toJsonDict())

{u'eventDate': None,
 u'familyLevelQuestions': {u'additionalComments': None,
                           u'caseSolvedFamily': None,
                           u'segregationQuestion': None},
 u'reporter': None,
 u'variantGroupLevelQuestions': None}


Passing this json to the `validate` method performs schema validation on the object

In [4]:
eq.validate(eq.toJsonDict())

False

This tells us that `eq` is not a valid version 3 `RareDiseaseExitQuestionnaire` object

Passing `verbose=True` to the method returns a list of messages that describe what is invalid about the object

In [5]:
eq.validate(eq.toJsonDict(), verbose=True).messages[1]

u'Class: [RareDiseaseExitQuestionnaire] expects field: [eventDate] with schema type: ["string"] but received value: [None]'

In [6]:
eq.eventDate = "2018-07-16"

In [7]:
eq.validate(eq.toJsonDict(), verbose=True).messages[1]

u'Class: [RareDiseaseExitQuestionnaire] expects field: [reporter] with schema type: ["string"] but received value: [None]'

In [8]:
eq.reporter = "Greg Lever"

In [9]:
eq.validate(eq.toJsonDict(), verbose=True).messages[2]

u'Class: [FamilyLevelQuestions] expects field: [caseSolvedFamily] with schema type: [{"symbols": ["yes", "no", "partially", "unknown"], "namespace": "org.gel.models.report.avro", "type": "enum", "name": "CaseSolvedFamily"}] but received value: [None]'

This tells us that the `famileLevelQuestions` field has it's own class. Documentation for it can be seen here:

http://gelreportmodels.genomicsengland.co.uk/html_schemas/org.gel.models.report.avro/3.0.0/ExitQuestionnaire.html#/schema/org.gel.models.report.avro.FamilyLevelQuestions

And can be constructed like so:

In [10]:
from protocols.reports_3_0_0 import FamilyLevelQuestions as FLQs

In [11]:
flqs = FLQs()

In [12]:
flqs.validate(flqs.toJsonDict(), verbose=True).messages[2]

u'Class: [FamilyLevelQuestions] expects field: [caseSolvedFamily] with schema type: [{"symbols": ["yes", "no", "partially", "unknown"], "namespace": "org.gel.models.report.avro", "type": "enum", "name": "CaseSolvedFamily"}] but received value: [None]'

In [13]:
flqs.caseSolvedFamily = "no"

In [14]:
flqs.validate(flqs.toJsonDict(), verbose=True).messages[2]

u'Class: [FamilyLevelQuestions] expects field: [segregationQuestion] with schema type: [{"symbols": ["yes", "no"], "namespace": "org.gel.models.report.avro", "type": "enum", "name": "SegregationQuestion"}] but received value: [None]'

In [15]:
flqs.segregationQuestion = "no"

In [17]:
flqs.validate(flqs.toJsonDict(), verbose=True).messages[1]

u'Class: [FamilyLevelQuestions] expects field: [additionalComments] with schema type: ["string"] but received value: [None]'

In [18]:
flqs.additionalComments = ""

In [19]:
flqs.validate(flqs.toJsonDict(), verbose=True).result

True

Now that we have a valid `FamilyLevelQuestions` object we can assign it to the `familyLevelQuestions` field of the `RareDiseaseExitQuestionnaire`

In [20]:
eq.familyLevelQuestions = flqs

In [ ]:
eq.validate(eq.toJsonDict())

In [21]:
eq.validate(eq.toJsonDict(), verbose=True).result

False

We are specifically dealing with negative cases here so `variantGroupLevelQuestions` can be an empty list

In [22]:
eq.variantGroupLevelQuestions = []

In [23]:
eq.validate(eq.toJsonDict(), verbose=True).result

True

In the future when we will want to programatically send positive cases, the Variant Group Level Questions model is documented here: http://gelreportmodels.genomicsengland.co.uk/html_schemas/org.gel.models.report.avro/3.0.0/ExitQuestionnaire.html#/schema/schemas%2FAVPRs%2Fbuild%2FExitQuestionnaire.avpr/org.gel.models.report.avro.VariantGroupLevelQuestions

The objects can also be instantiated in this way:

In [ ]:
from protocols.reports_3_0_0 import RareDiseaseExitQuestionnaire as EQ
from protocols.reports_3_0_0 import FamilyLevelQuestions as FLQs

flqs = FLQs(
    caseSolvedFamily = "no",
    segregationQuestion = "no",
    additionalComments = "",
)
eq = EQ(
    eventDate="2018-07-16",
    reporter="Greg Lever",
    familyLevelQuestions=flqs,
    variantGroupLevelQuestions=[],
)

In [24]:
eq.validate(eq.toJsonDict())

True

In [25]:
pprint(eq.toJsonDict())

{u'eventDate': '2018-07-16',
 u'familyLevelQuestions': {u'additionalComments': '',
                           u'caseSolvedFamily': 'no',
                           u'segregationQuestion': 'no'},
 u'reporter': 'Greg Lever',
 u'variantGroupLevelQuestions': []}


### Now the Exit Questionnaire payload is ready to be sent to the CIP API

`get_authenticated_header` is a simple method for authenticating your credentials with the CIP API, which will return you authenticated headers to supply with your HTTP requests.

In [28]:
import os
import requests


def get_authenticated_header(url, username):
    
    url += "{endpoint}"
    auth_endpoint = "get-token/"
    import getpass
    password = getpass.getpass()

    irl_response = requests.post(
        url=url.format(endpoint=auth_endpoint),
        json=dict(
            username=username,
            password=password,
        ),
    )
    irl_response_json = irl_response.json()
    token = irl_response_json.get('token')

    auth_header = {
        'Accept': 'application/json',
        "Authorization": "JWT {token}".format(token=token),
    }
    return auth_header

You will be prompted to enter your CIP API password here

In [29]:
import requests
cip_api_url = "https://cipapi-test-tng.gel.zone/api/2/"
auth_header = get_authenticated_header(url=cip_api_url, username="glever")

········


I have a case in mind, with ID 2 and version 2

In [33]:
"""
GET /api/2/interpretation-request/{ir_id}/{ir_version}/ 
"""
endpoint = "interpretation-request/{ir_id}/{ir_version}/".format(
        ir_id=2, ir_version=2,
)
url = cip_api_url + endpoint
print url

https://cipapi-test-tng.gel.zone/api/2/interpretation-request/2/2/


In [34]:
response = requests.get(url=url, headers=auth_header)

In [35]:
response.status_code

200

In [36]:
response.json().keys()

[u'files',
 u'number_of_samples',
 u'version',
 u'gel_tiering_qc_outcome',
 u'clinical_report',
 u'family_id',
 u'labkey_links',
 u'sample_type',
 u'proband',
 u'status',
 u'case_priority',
 u'assembly',
 u'tags',
 u'cva_variants_status',
 u'case_id',
 u'last_status',
 u'interpretation_request_data',
 u'cva_variants_transaction_id',
 u'created_at',
 u'interpretation_request_id',
 u'cohort_id',
 u'cip',
 u'interpreted_genome']

The data returned in the interpretation_request_data.json_request corresponds with the Interpretation Request GelReportModels model and an object could be created using that data accordingly. 

In [37]:
len(response.json().get("clinical_report"))

1

This part of the json response contains one item per each summary of findings. Therefore we know the latest summary of findings for this case is version 1

In [38]:
"""
PUT /api/2/exit-questionnaire/{ir_id}/{ir_version}/{clinical_report_version}/ 
"""
endpoint = "exit-questionnaire/{ir_id}/{ir_version}/{clinical_report_version}/".format(
        ir_id=2, ir_version=2, clinical_report_version=1
)
url = cip_api_url + endpoint
print url

https://cipapi-test-tng.gel.zone/api/2/exit-questionnaire/2/2/1/


We now want to PUT the Exit Questionnaire json payload

In [39]:
response = requests.put(url=url, headers=auth_header, json=eq.toJsonDict())

A 200 response will be returned if successful  and the content of the response will show the data we sent, along with metadata concering the user who made the request and when they did so. 

In [40]:
response.status_code

200

In [41]:
response.content

'{"exit_questionnaire_data":{"familyLevelQuestions":{"caseSolvedFamily":"no","segregationQuestion":"no","additionalComments":""},"reporter":"glever","eventDate":"2018-07-16","variantGroupLevelQuestions":[]},"user":"glever","created_at":"2018-07-16T08:27:38.877131Z","cva_status":"not_sent","cva_transaction_id":0}'